# Stable Baselines3 Tutorial - Gym wrappers, saving and loading models

Github repo: https://github.com/araffin/rl-tutorial-jnrr19/tree/sb3/

Stable-Baselines3: https://github.com/DLR-RM/stable-baselines3

Documentation: https://stable-baselines3.readthedocs.io/en/master/

RL Baselines3 zoo: https://github.com/DLR-RM/rl-baselines3-zoo


## Introduction

In this notebook, you will learn how to use *Gym Wrappers* which allow to do monitoring, normalization, limit the number of steps, feature augmentation, ...


You will also see the *loading* and *saving* functions, and how to read the outputed files for possible exporting.

## Install Dependencies and Stable Baselines3 Using Pip

In [1]:
!apt install swig
!pip install stable-baselines3[extra]

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 37 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 0s (9,050 kB/s)
Selecting previously unselected package swig3.0.
(Reading database ... 155219 files and directories currently installed.)
Preparing to unpack .../swig3.0_3.0.12-1_amd64.deb ...
Unpacking swig3.0 (3.0.12-1) ...
Selecting previously unselected package swig.
Preparing to unpack .../swig_3.0.12-1_amd64.deb ...
Unpackin

In [2]:
import gym
from stable_baselines3 import A2C, SAC, PPO, TD3

# Saving and loading

Saving and loading stable-baselines models is straightforward: you can directly call `.save()` and `.load()` on the models.

In [3]:
import os

# Create save dir
save_dir = "/tmp/gym/"
os.makedirs(save_dir, exist_ok=True)

model = PPO('MlpPolicy', 'Pendulum-v0', verbose=0).learn(8000)
# The model will be saved under PPO_tutorial.zip
model.save(save_dir + "/PPO_tutorial")

# sample an observation from the environment
obs = model.env.observation_space.sample()

# Check prediction before saving
print("pre saved", model.predict(obs, deterministic=True))

del model # delete trained model to demonstrate loading

loaded_model = PPO.load(save_dir + "/PPO_tutorial")
# Check that the prediction is the same after loading (for the same observation)
print("loaded", loaded_model.predict(obs, deterministic=True))

pre saved (array([-0.05292664], dtype=float32), None)
loaded (array([-0.05292664], dtype=float32), None)


Saving in stable-baselines is quite powerful, as you save the training hyperparameters, with the current weights. This means in practice, you can simply load a custom model, without redefining the parameters, and continue learning.

The loading function can also update the model's class variables when loading.

In [4]:
import os
from stable_baselines3.common.vec_env import DummyVecEnv

# Create save dir
save_dir = "/tmp/gym/"
os.makedirs(save_dir, exist_ok=True)

model = A2C('MlpPolicy', 'Pendulum-v0', verbose=0, gamma=0.9, n_steps=20).learn(8000)
# The model will be saved under A2C_tutorial.zip
model.save(save_dir + "/A2C_tutorial")

del model # delete trained model to demonstrate loading

# load the model, and when loading set verbose to 1
loaded_model = A2C.load(save_dir + "/A2C_tutorial", verbose=1)

# show the save hyperparameters
print("loaded:", "gamma =", loaded_model.gamma, "n_steps =", loaded_model.n_steps)

# as the environment is not serializable, we need to set a new instance of the environment
loaded_model.set_env(DummyVecEnv([lambda: gym.make('Pendulum-v0')]))
# and continue training
loaded_model.learn(8000)

loaded: gamma = 0.9 n_steps = 20
------------------------------------
| time/                 |          |
|    fps                | 1532     |
|    iterations         | 100      |
|    time_elapsed       | 1        |
|    total_timesteps    | 2000     |
| train/                |          |
|    entropy_loss       | -1.42    |
|    explained_variance | 0.000453 |
|    learning_rate      | 0.0007   |
|    n_updates          | 499      |
|    policy_loss        | -39.3    |
|    std                | 1        |
|    value_loss         | 809      |
------------------------------------
-------------------------------------
| time/                 |           |
|    fps                | 1535      |
|    iterations         | 200       |
|    time_elapsed       | 2         |
|    total_timesteps    | 4000      |
| train/                |           |
|    entropy_loss       | -1.42     |
|    explained_variance | -8.11e-06 |
|    learning_rate      | 0.0007    |
|    n_updates          | 599   

# Gym and VecEnv wrappers

## Anatomy of a gym wrapper

A gym wrapper follows the [gym](https://stable-baselines.readthedocs.io/en/master/guide/custom_env.html) interface: it has a `reset()` and `step()` method.

Because a wrapper is *around* an environment, we can access it with `self.env`, this allow to easily interact with it without modifying the original env.
There are many wrappers that have been predefined, for a complete list refer to [gym documentation](https://github.com/openai/gym/tree/master/gym/wrappers)

In [5]:
class CustomWrapper(gym.Wrapper):
  """
  :param env: (gym.Env) Gym environment that will be wrapped
  """
  def __init__(self, env):
    # Call the parent constructor, so we can access self.env later
    super(CustomWrapper, self).__init__(env)
  
  def reset(self):
    """
    Reset the environment 
    """
    obs = self.env.reset()
    return obs

  def step(self, action):
    """
    :param action: ([float] or int) Action taken by the agent
    :return: (np.ndarray, float, bool, dict) observation, reward, is the episode over?, additional informations
    """
    obs, reward, done, info = self.env.step(action)
    return obs, reward, done, info


## First example: limit the episode length

One practical use case of a wrapper is when you want to limit the number of steps by episode, for that you will need to overwrite the `done` signal when the limit is reached. It is also a good practice to pass that information in the `info` dictionnary.

In [6]:
class TimeLimitWrapper(gym.Wrapper):
  """
  :param env: (gym.Env) Gym environment that will be wrapped
  :param max_steps: (int) Max number of steps per episode
  """
  def __init__(self, env, max_steps=100):
    # Call the parent constructor, so we can access self.env later
    super(TimeLimitWrapper, self).__init__(env)
    self.max_steps = max_steps
    # Counter of steps per episode
    self.current_step = 0
  
  def reset(self):
    """
    Reset the environment 
    """
    # Reset the counter
    self.current_step = 0
    return self.env.reset()

  def step(self, action):
    """
    :param action: ([float] or int) Action taken by the agent
    :return: (np.ndarray, float, bool, dict) observation, reward, is the episode over?, additional informations
    """
    self.current_step += 1
    obs, reward, done, info = self.env.step(action)
    # Overwrite the done signal when 
    if self.current_step >= self.max_steps:
      done = True
      # Update the info dict to signal that the limit was exceeded
      info['time_limit_reached'] = True
    return obs, reward, done, info


#### Test the wrapper

In [7]:
from gym.envs.classic_control.pendulum import PendulumEnv

# Here we create the environment directly because gym.make() already wrap the environement in a TimeLimit wrapper otherwise
env = PendulumEnv()
# Wrap the environment
env = TimeLimitWrapper(env, max_steps=100)

In [8]:
obs = env.reset()
done = False
n_steps = 0
while not done:
  # Take random actions
  random_action = env.action_space.sample()
  obs, reward, done, info = env.step(random_action)
  n_steps += 1

print(n_steps, info)

100 {'time_limit_reached': True}


In practice, `gym` already have a wrapper for that named `TimeLimit` (`gym.wrappers.TimeLimit`) that is used by most environments.

## Second example: normalize actions

It is usually a good idea to normalize observations and actions before giving it to the agent, this prevent [hard to debug issue](https://github.com/hill-a/stable-baselines/issues/473).

In this example, we are going to normalize the action space of *Pendulum-v0* so it lies in [-1, 1] instead of [-2, 2].

Note: here we are dealing with continuous actions, hence the `gym.Box` space

In [9]:
import numpy as np

class NormalizeActionWrapper(gym.Wrapper):
  """
  :param env: (gym.Env) Gym environment that will be wrapped
  """
  def __init__(self, env):
    # Retrieve the action space
    action_space = env.action_space
    assert isinstance(action_space, gym.spaces.Box), "This wrapper only works with continuous action space (spaces.Box)"
    # Retrieve the max/min values
    self.low, self.high = action_space.low, action_space.high

    # We modify the action space, so all actions will lie in [-1, 1]
    env.action_space = gym.spaces.Box(low=-1, high=1, shape=action_space.shape, dtype=np.float32)

    # Call the parent constructor, so we can access self.env later
    super(NormalizeActionWrapper, self).__init__(env)
  
  def rescale_action(self, scaled_action):
      """
      Rescale the action from [-1, 1] to [low, high]
      (no need for symmetric action space)
      :param scaled_action: (np.ndarray)
      :return: (np.ndarray)
      """
      return self.low + (0.5 * (scaled_action + 1.0) * (self.high -  self.low))

  def reset(self):
    """
    Reset the environment 
    """
    # Reset the counter
    return self.env.reset()

  def step(self, action):
    """
    :param action: ([float] or int) Action taken by the agent
    :return: (np.ndarray, float, bool, dict) observation, reward, is the episode over?, additional informations
    """
    # Rescale action from [-1, 1] to original [low, high] interval
    rescaled_action = self.rescale_action(action)
    obs, reward, done, info = self.env.step(rescaled_action)
    return obs, reward, done, info


#### Test before rescaling actions

In [10]:
original_env = gym.make("Pendulum-v0")

print(original_env.action_space.low)
for _ in range(10):
  print(original_env.action_space.sample())

[-2.]
[-1.8865008]
[0.76111114]
[-1.2962222]
[-0.14304268]
[0.7447886]
[-1.4128095]
[-1.5202788]
[1.5071152]
[-1.0076494]
[1.4562893]


#### Test the NormalizeAction wrapper

In [11]:
env = NormalizeActionWrapper(gym.make("Pendulum-v0"))

print(env.action_space.low)

for _ in range(10):
  print(env.action_space.sample())

[-1.]
[-0.7095748]
[0.02797176]
[0.5938129]
[-0.01302343]
[0.17248467]
[-0.6452554]
[0.94569093]
[-0.5845174]
[0.6165716]
[-0.11070762]


#### Test with a RL algorithm

We are going to use the Monitor wrapper of stable baselines, wich allow to monitor training stats (mean episode reward, mean episode length)

In [12]:
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv

In [13]:
env = Monitor(gym.make('Pendulum-v0'))
env = DummyVecEnv([lambda: env])

In [14]:
model = A2C("MlpPolicy", env, verbose=1).learn(int(1000))

Using cpu device
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.47e+03 |
| time/                 |           |
|    fps                | 920       |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.44     |
|    explained_variance | -0.00501  |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -39.1     |
|    std                | 1.02      |
|    value_loss         | 1.34e+03  |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.37e+03 |
| time/                 |           |
|    fps                | 932       |
|    iterations         | 200       |
|    time_elapsed       | 1      

With the action wrapper

In [15]:
normalized_env = Monitor(gym.make('Pendulum-v0'))
# Note that we can use multiple wrappers
normalized_env = NormalizeActionWrapper(normalized_env)
normalized_env = DummyVecEnv([lambda: normalized_env])

In [16]:
model_2 = A2C("MlpPolicy", normalized_env, verbose=1).learn(int(1000))

Using cpu device
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.09e+03 |
| time/                 |           |
|    fps                | 929       |
|    iterations         | 100       |
|    time_elapsed       | 0         |
|    total_timesteps    | 500       |
| train/                |           |
|    entropy_loss       | -1.42     |
|    explained_variance | 0.0341    |
|    learning_rate      | 0.0007    |
|    n_updates          | 99        |
|    policy_loss        | -4.01     |
|    std                | 0.999     |
|    value_loss         | 7.26      |
-------------------------------------
-------------------------------------
| rollout/              |           |
|    ep_len_mean        | 200       |
|    ep_rew_mean        | -1.25e+03 |
| time/                 |           |
|    fps                | 924       |
|    iterations         | 200       |
|    time_elapsed       | 1      

## Additional wrappers: VecEnvWrappers

In the same vein as gym wrappers, stable baselines provide wrappers for `VecEnv`. Among the different that exist (and you can create your own), you should know: 

- VecNormalize: it computes a running mean and standard deviation to normalize observation and returns
- VecFrameStack: it stacks several consecutive observations (useful to integrate time in the observation, e.g. sucessive frame of an atari game)

More info in the [documentation](https://stable-baselines3.readthedocs.io/en/master/guide/vec_envs.html#wrappers)

Note: when using `VecNormalize` wrapper, you must save the running mean and std along with the model, otherwise you will not get proper results when loading the agent again. If you use the [rl zoo](https://github.com/DLR-RM/rl-baselines3-zoo), this is done automatically

In [17]:
from stable_baselines3.common.vec_env import VecNormalize, VecFrameStack

env = DummyVecEnv([lambda: gym.make("Pendulum-v0")])
normalized_vec_env = VecNormalize(env)

In [18]:
obs = normalized_vec_env.reset()
for _ in range(10):
  action = [normalized_vec_env.action_space.sample()]
  obs, reward, _, _ = normalized_vec_env.step(action)
  print(obs, reward)

[[-0.54705644 -0.12768991  0.9998632 ]] [-10.]
[[-1.3049014 -0.7458486  1.0757202]] [-2.0198078]
[[-1.5993044 -1.4638258  1.3217945]] [-1.2708998]
[[-1.7483342 -1.7895812  1.4796199]] [-0.99464875]
[[-1.8062567 -1.9423994  1.4834249]] [-0.87714076]
[[-1.8123285 -2.0375621  1.4577675]] [-0.8079896]
[[-1.7793765 -2.1048453  1.447266 ]] [-0.758458]
[[-1.6930909 -2.135008   1.3786477]] [-0.72255445]
[[-1.5511655 -2.1546626  1.3992189]] [-0.68292195]
[[-1.3239248 -2.11952    1.2115979]] [-0.65953004]


## Exercise: code you own monitor wrapper

Now that you know how does a wrapper work and what you can do with it, it's time to experiment.

The goal here is to create a wrapper that will monitor the training progress, storing both the episode reward (sum of reward for one episode) and episode length (number of steps in for the last episode).

You will return those values using the `info` dict after each end of episode.

In [20]:
class MyMonitorWrapper(gym.Wrapper):
  """
  :param env: (gym.Env) Gym environment that will be wrapped
  """
  def __init__(self, env):
    # Call the parent constructor, so we can access self.env later
    super(MyMonitorWrapper, self).__init__(env)
    # === YOUR CODE HERE ===#
    # Initialize the variables that will be used
    # to store the episode length and episode reward

    # ====================== #
  
  def reset(self):
    """
    Reset the environment 
    """
    obs = self.env.reset()
    # === YOUR CODE HERE ===#
    # Reset the variables

    # ====================== #
    return obs

  def step(self, action):
    """
    :param action: ([float] or int) Action taken by the agent
    :return: (np.ndarray, float, bool, dict) observation, reward, is the episode over?, additional informations
    """
    obs, reward, done, info = self.env.step(action)
    # === YOUR CODE HERE ===#
    # Update the current episode reward and episode length

    # ====================== #

    if done:
      # === YOUR CODE HERE ===#
      # Store the episode length and episode reward in the info dict
      print(info)  # added just to see what it looks like before modifying
      # ====================== #
    
    return obs, reward, done, info

#### Test your wrapper

In [21]:
# To use LunarLander, you need to install box2d box2d-kengz (pip) and swig (apt-get)
!pip install box2d box2d-kengz

     |████████████████████████████████| 1.3 MB 14.0 MB/s 
     |████████████████████████████████| 425 kB 70.4 MB/s 
  Created wheel for box2d-kengz: filename=Box2D_kengz-2.3.3-cp37-cp37m-linux_x86_64.whl size=2053074 sha256=906863e3da1bf7d51ec2c5561b26948bac10d2cd3ca235f0050003ae9bd3f8ed
  Stored in directory: /root/.cache/pip/wheels/50/6d/6a/6ff76731fd9e8efbd1cdc6111e98b2dd0f1872184d7c28939c
Successfully built box2d-kengz


In [22]:
env = gym.make("LunarLander-v2")
# === YOUR CODE HERE ===#
# Wrap the environment

# Reset the environment

# Take random actions in the enviromnent and check
# that it returns the correct values after the end of each episode

# ====================== #

 # Conclusion
 
 In this notebook, we have seen:
 - how to easily save and load a model
 - what is wrapper and what we can do with it
 - how to create your own wrapper

## Wrapper Bonus: changing the observation space: a wrapper for episode of fixed length

In [ ]:
from gym.wrappers import TimeLimit

class TimeFeatureWrapper(gym.Wrapper):
    """
    Add remaining time to observation space for fixed length episodes.
    See https://arxiv.org/abs/1712.00378 and https://github.com/aravindr93/mjrl/issues/13.

    :param env: (gym.Env)
    :param max_steps: (int) Max number of steps of an episode
        if it is not wrapped in a TimeLimit object.
    :param test_mode: (bool) In test mode, the time feature is constant,
        equal to zero. This allow to check that the agent did not overfit this feature,
        learning a deterministic pre-defined sequence of actions.
    """
    def __init__(self, env, max_steps=1000, test_mode=False):
        assert isinstance(env.observation_space, gym.spaces.Box)
        # Add a time feature to the observation
        low, high = env.observation_space.low, env.observation_space.high
        low, high= np.concatenate((low, [0])), np.concatenate((high, [1.]))
        env.observation_space = gym.spaces.Box(low=low, high=high, dtype=np.float32)

        super(TimeFeatureWrapper, self).__init__(env)

        if isinstance(env, TimeLimit):
            self._max_steps = env._max_episode_steps
        else:
            self._max_steps = max_steps
        self._current_step = 0
        self._test_mode = test_mode

    def reset(self):
        self._current_step = 0
        return self._get_obs(self.env.reset())

    def step(self, action):
        self._current_step += 1
        obs, reward, done, info = self.env.step(action)
        return self._get_obs(obs), reward, done, info

    def _get_obs(self, obs):
        """
        Concatenate the time feature to the current observation.

        :param obs: (np.ndarray)
        :return: (np.ndarray)
        """
        # Remaining time is more general
        time_feature = 1 - (self._current_step / self._max_steps)
        if self._test_mode:
            time_feature = 1.0
        # Optionnaly: concatenate [time_feature, time_feature ** 2]
        return np.concatenate((obs, [time_feature]))

## Going further - Saving format 

The format for saving and loading models is a zip-archived JSON dump and NumPy zip archive of the arrays:
```
saved_model.zip/
├── data              JSON file of class-parameters (dictionary)
├── parameter_list    JSON file of model parameters and their ordering (list)
├── parameters        Bytes from numpy.savez (a zip file of the numpy arrays). ...
    ├── ...           Being a zip-archive itself, this object can also be opened ...
        ├── ...       as a zip-archive and browsed.
```

## Save and find 

In [ ]:
# Create save dir
save_dir = "/tmp/gym/"
os.makedirs(save_dir, exist_ok=True)

model = PPO('MlpPolicy', 'Pendulum-v0', verbose=0).learn(8000)
model.save(save_dir + "/PPO_tutorial")

In [ ]:
!ls /tmp/gym/PPO_tutorial*

In [ ]:
import zipfile

archive = zipfile.ZipFile("/tmp/gym/PPO_tutorial.zip", 'r')
for f in archive.filelist:
  print(f.filename)

## Exporting saved models

And finally some futher reading for those who want to export to tensorflowJS or Java.

https://stable-baselines.readthedocs.io/en/master/guide/export.html